# Azure AI Agent service - MCP support

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol

Supported regions: westus, westus2, uaenorth, southindia and switzerlandnorth

Install required packages:

`pip install gradio python-dotenv azure-identity azure-ai-projects>=1.0.0b12 azure-ai-agents>=1.1.0b4 --pre`

In [3]:
import os
import gradio as gr
import json
import re
import sys
import time

from azure.ai.agents.models import ListSortOrder, MessageTextContent, McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [4]:
sys.version

'3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]'

## 1. Settings

In [ ]:
load_dotenv("azure.env")

PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT2")

In [6]:
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT,
                                 credential=DefaultAzureCredential())

# Convenience handle for the Agents sub‑client
agents_client = project_client.agents

In [ ]:
model = "gpt-4o"

## 2. Example

In [8]:
# With this prefix 'https://gitmcp.io/' you can instantly create a remote RCP server from any GitHub repo
github_url = "https://github.com/Azure-Samples/ai-foundry-agents-samples"

get_substr = re.search(r'\.com/(.*)', github_url)
substr = get_substr.group(1)
mcp_server_url = f"https://gitmcp.io/{substr}"

mcp_server_url

'https://gitmcp.io/Azure-Samples/ai-foundry-agents-samples'

In [9]:
prompt = "Any python code for a mcp server with Azure AI Agent?"

In [10]:
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT,
                                 credential=DefaultAzureCredential())

# MCP tool definition
mcp_tool = McpTool(
    server_label="github",
    server_url=mcp_server_url
    #allowed_tools=[],          # Optional initial allow‑list
)

# Agent creation
agent = agents_client.create_agent(
    model=model,
    name="agent-mcp",
    instructions="You are an AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
    tools=mcp_tool.definitions
)
print(f"🎉 Created agent, agent ID: {agent.id}")
print(f"✨ MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

# Thread creation
thread = agents_client.threads.create()
print(f"🧵 Created thread, thread ID: {thread.id}")

# Message creation
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
print(f"💬 Created message, message ID: {message.id}")

mcp_tool.set_approval_mode("never")          # Disable human approval

# Run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(2)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"⏳ Run status: {run.status}")
if run.status == "failed":
    print(f"❌ Run error: {run.last_error}")

# Run steps
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
print()

for step in run_steps:
    print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"🛠️ Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=5))


🎉 Created agent, agent ID: asst_VBtLPkeDbb5q6EQFHYbVgiqU
✨ MCP Server: github at https://gitmcp.io/Azure-Samples/ai-foundry-agents-samples
🧵 Created thread, thread ID: thread_nxwDB3f0Mynf680AQCddzoYr
💬 Created message, message ID: msg_c61cRR2cqoI0SvBYnFv6mISA
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.COMPLETED

🔄 Run step: step_troq6NJjKe8jkkFJZDvFhgPS, status: RunStepStatus.COMPLETED, type: RunStepType

In [11]:
messages = agents_client.messages.list(thread_id=thread.id,
                                               order=ListSortOrder.ASCENDING)

for item in messages:
    last_message_content = item.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{item.role}: {last_message_content.text.value}")

MessageRole.USER: Any python code for a mcp server with Azure AI Agent?
MessageRole.AGENT: Yes, there are working Python examples for an MCP (Model Context Protocol) server that integrates with an Azure AI Agent in the [Azure-Samples/ai-foundry-agents-samples](https://github.com/Azure-Samples/ai-foundry-agents-samples) repository.

### Example: Streamable HTTP MCP Server (Weather Agent)

**Code location:**  
`examples/mcp/streamable-http/mcp-server/weather.py` ([view on GitHub](https://github.com/Azure-Samples/ai-foundry-agents-samples/blob/main/examples/mcp/streamable-http/mcp-server/weather.py))

This file implements an MCP server using [FastMCP](https://pypi.org/project/mcp/) and provides weather querying tools as MCP agent tools.

#### Main parts of the `weather.py` MCP Server

```python
from typing import Any
import httpx
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("weather", port=8001, host="0.0.0.0", stateless_http=True)

# Tool implementati

## Another example

In [12]:
# With this prefix 'https://gitmcp.io/' you can instantly create a remote RCP server from any GitHub repo
github_url = "https://github.com/openai/openai-python"

get_substr = re.search(r'\.com/(.*)', github_url)
substr = get_substr.group(1)
mcp_server_url = f"https://gitmcp.io/{substr}"

mcp_server_url

'https://gitmcp.io/openai/openai-python'

In [13]:
prompt = "Show me some python code to use whisper"

In [14]:
# Agent creation
agent = agents_client.create_agent(
    model=model,
    name="agent-mcp",
    instructions="You are an AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
    tools=mcp_tool.definitions
)
print(f"🎉 Created agent, agent ID: {agent.id}")
print(f"✨ MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

# Thread creation
thread = agents_client.threads.create()
print(f"🧵 Created thread, thread ID: {thread.id}")

# Message creation
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
print(f"💬 Created message, message ID: {message.id}")

mcp_tool.set_approval_mode("never")          # Disable human approval

# Run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(2)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"⏳ Run status: {run.status}")
if run.status == "failed":
    print(f"❌ Run error: {run.last_error}")

# Run steps
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
print()

for step in run_steps:
    print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"🛠️ Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=5))


🎉 Created agent, agent ID: asst_T3ZyQoulo1muwcRwE5KVp4xT
✨ MCP Server: github at https://gitmcp.io/Azure-Samples/ai-foundry-agents-samples
🧵 Created thread, thread ID: thread_ItCjtzvfu3mVkvPXDUanxHRX
💬 Created message, message ID: msg_lagFQkV60EsgxFCBoVTSvSjf
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.COMPLETED

🔄 Run step: step_0wkDolp9rwqZZvI1fJXxNhI6, status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION


In [15]:
messages = agents_client.messages.list(thread_id=thread.id,
                                               order=ListSortOrder.ASCENDING)

for item in messages:
    last_message_content = item.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{item.role}: {last_message_content.text.value}")

MessageRole.USER: Show me some python code to use whisper
MessageRole.AGENT: Sure! Here’s a basic Python example to use OpenAI’s Whisper model for speech-to-text transcription. This uses the official [openai/whisper](https://github.com/openai/whisper) Python library, which you can install via pip.

### Step 1: Install Whisper

First, install the package:

```bash
pip install -U openai-whisper
```

Make sure you also have ffmpeg installed, since Whisper uses it for audio processing:

```bash
# For Ubuntu/Debian
sudo apt-get install ffmpeg

# For MacOS (with Homebrew)
brew install ffmpeg
```

### Step 2: Basic Usage Example

Save the following script as, for example, `transcribe.py`:

```python
import whisper

# Load the pre-trained Whisper model (base, tiny, small, medium, large)
model = whisper.load_model("base")

# Transcribe an audio file
result = model.transcribe("your-audio-file.mp3")

print("Transcribed text:")
print(result["text"])
```

Replace `"your-audio-file.mp3"` with the pa

## 3. Webapp

In [16]:
def agent_mcp_fn(github_url, prompt):
    """
    Creates an AI agent to interact with a specified GitHub repository and processes a given prompt.

    Args:
        github_url (str): The URL of the GitHub repository.
        prompt (str): The prompt to be processed by the agent.

    Returns:
        tuple: A tuple containing:
            - str: The concatenated responses from the agent.
            - str: The URLs extracted from the agent's responses.

    Raises:
        ValueError: If the GitHub URL format is invalid.
        RuntimeError: If the agent's run fails.
        Exception: For any other errors that occur during execution.
    """
    try:
        get_substr = re.search(r'\.com/(.*)', github_url)

        if get_substr:
            substr = get_substr.group(1)
        else:
            raise ValueError("Invalid URL format.")
        
        mcp_server_url = f"https://gitmcp.io/{substr}"

        # Agent creation
        agent = agents_client.create_agent(
            model=model,
            name="agent-mcp",
            instructions="You are an AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
            tools=mcp_tool.definitions
        )
        print(f"🎉 Created agent, agent ID: {agent.id}")
        print(f"✨ MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

        # Thread creation
        thread = agents_client.threads.create()
        print(f"🧵 Created thread, thread ID: {thread.id}")

        # Message creation
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=prompt,
        )
        print(f"💬 Created message, message ID: {message.id}")

        mcp_tool.set_approval_mode("never")  # Disable human approval

        # Run
        run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(2)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
            print(f"⏳ Run status: {run.status}")
        if run.status == "failed":
            raise RuntimeError(f"❌ Run error: {run.last_error}")

        # Run steps
        run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
        
        for step in run_steps:
            print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
            if step.type == "tool_calls":
                print(f"🛠️ Tool call details:")
                for tool_call in step.step_details.tool_calls:
                    print(json.dumps(tool_call.as_dict(), indent=5))

        messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

        answers = []
        for item in messages:
            last_message_content = item.content[-1]
            if isinstance(last_message_content, MessageTextContent):
                answers.append(f"{item.role}: {last_message_content.text.value}")
        
        url_template = r'(https?://[^\s]+|www\.[^\s]+)'
        urls = re.findall(url_template, "\n".join(answers))
        url_links = ' '.join(urls).replace(')', '').replace(' ', '\n')

        return "\n".join(answers), url_links
        
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [17]:
def build_webapp():
    with gr.Blocks(title="Azure AI Foundry Agent") as demo:
        gr.Markdown("# 🤖 Azure AI Foundry Agent")
        gr.Markdown("## Enter a GitHub repository URL and your prompt to interact with the MCP server using Azure AI agent.")

        with gr.Row():
            github_url = gr.Textbox(label="🔗 GitHub Repository URL", placeholder="e.g. https://github.com/org/repo")
            prompt = gr.Textbox(label="📝 Prompt", placeholder="What does this GitHub repo do?", lines=2)

        with gr.Row():
            results_output = gr.Textbox(label="📄 Agent response", lines=25, interactive=False)
            links_output = gr.Textbox(label="🔍 Links", lines=5, interactive=False)

        run_button = gr.Button("🚀 Run Agent")
        run_button.click(fn=agent_mcp_fn, inputs=[github_url, prompt], outputs=[results_output, links_output])

    return demo


webapp = build_webapp()
webapp.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


🎉 Created agent, agent ID: asst_PTiz61qxSQXpphxdOYJSnR6Y
✨ MCP Server: github at https://gitmcp.io/Azure-Samples/ai-foundry-agents-samples
🧵 Created thread, thread ID: thread_wEtZanI86shs5XwU6B0TNimo
💬 Created message, message ID: msg_vghPyhrUOmvg78mVysEdahXY
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.COMPLETED
🔄 Run step: step_2cpREXqO2f8IyIB3x90sPWzK, status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION
🔄 Run step: step_4Li8qJaLhNjTgp3EMdFBN177, status: RunStepStatus.COMPLETED, type: RunStepType.TOOL_CALLS
🛠️ Tool call details:
{
     "id": "call_884S2maLr3zb0e3dIXJoYm6a",
     "type": "mcp",
     "arguments": "{}",
     "name": "fetch_ai_foundry_docs",
     "server_label": "github",
     "output": "# Azu